# Project: A/B Test a New Menu Launch (Part1)

### by Sooyeon Won

### Keywords
- Analytical Framework
- Data Cleaning 
- Data Exploration 
- A/B tests Design and Analysis 

## Table of Contents
<ul>
<li><a href="#Intro">Introduction (Part1)</a></li>
<li><a href="#Wrangling">Data Wrangling (Part1)</a></li>
<li><a href="#Exploration">Data Exploration (Part2)</a></li>
<li><a href="#Analysis">Data Analysis (Part3)</a></li>
<li><a href="#Conclusion">Conclusion (Part3)</a></li>
</ul>

<a id='Intro'></a>
## Introduction

This project is originally from [Udacity Predictive Analytics Nanodegree](https://www.udacity.com/course/predictive-analytics-for-business-nanodegree--nd008). Initially, this project is analysed using [Alteryx](Alteryx.com). As reviewing the previous lessons, I analysed the same project using python for this time. Though I followed the same workflow before, I couldn't reach the exact same results yet. However, the analysis will be continously updated. 


This analysis is regarding an A/B Test for deciding a Menu Launch. 
A chain of coffee shops(Round Roasters - A coffee restaurant in the United States of America) is considering launching a new menu. The executive team conducted a market test with a new menu and needs to figure whether the new menu can drive enough sales to offset the cost of marketing the new menu. <br>
 
To minimize risk, the management team decides to test the changes in two cities with new television advertising. Denver and Chicago cities were chosen to participate in this test because the stores in these two cities (or markets) perform similarly to all stores across the entire chain of stores; performance in these two markets would be a good proxy to predict how well the updated menu performs. <br>

The test ran for a period of 12 weeks **(2016-April-29 to 2016-July-21)** where five stores in each of the test markets offered the updated menu along with television advertising. The comparative period is the test period, but for last year **(2015-April-29 to 2015-July-21)**.


I design an A/B test and analyze the results of the experiment to determine whether the menu changes should be applied to all stores. 
The predicted impact to profitability should be enough to justify the increased marketing budget: at least 18% increase in profit growth compared to the comparative period while compared to the control stores; otherwise known as incremental lift. In the data, profit is represented in the gross_margin variable.

For the analysis, three different datasets are provided. 
- **(Transaction Data)** Transaction data for all stores from 2015-January-21 to 2016-August-18 
- **(All Stores Data)** A list of all Round Roasters stores 
- **(Treatment Stores Data)** A list of the 10 stores (5 in each market) that were used as test markets. 

Finally, I wrote up a recommendation on whether the chain should introduce the new menu. 



<a id='Wrangling'></a>
## Data Wrangling

### Data Assessment & Cleaning

1. Transaction Data
2. All Stores Data
3. Treatment Stores Data
4. Tidiness

In [1]:
# Import the relevant libraries 
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set()
sns.set(style="whitegrid", color_codes=True)

In [2]:
# Load the provided datasets
transaction_data = pd.read_csv('RoundRoastersTransactions.csv') # Transaction Data
treatment_stores = pd.read_csv('treatment-stores.csv')  # Treatment Stores Data
all_stores = pd.read_csv('round-roaster-stores.csv') # All Stores Data

### 1. Transaction Data
<br>
<li><a href="#assess_transaction"> Data Assessment (Transaction Data)</a></li>
<li><a href="#cleaning_transaction"> Data Cleaning (Transaction Data)</a></li>


In [3]:
transaction_data.head() # Glance at the dataset(1) 

,StoreID,Invoice Number,Invoice Date,SKU,Category,Product,QTY,Size,Gross Margin,Sales
0,10018,16296643,2015-01-21,1043,Espresso,Mocha,3,L,6.7365,14.97
1,10018,16296643,2015-01-21,2001,Pastry,Croissant,1,NaN,1.1000,2.75
2,10018,16297717,2015-01-21,1021,Espresso,Espresso,3,S,4.1850,8.37
3,10018,16297717,2015-01-21,1022,Espresso,Espresso,4,M,5.9800,11.96
4,10018,16297717,2015-01-21,1023,Espresso,Espresso,3,L,4.7850,9.57


In [4]:
transaction_data.shape  # Glance at the dataset(2). The dataset contains 4,332,333 datapoints with 10 fields.

(4332333, 10)

In [5]:
transaction_data.info()  # Check the datatype of each field and missing values in each field.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4332333 entries, 0 to 4332332
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   StoreID         int64  
 1   Invoice Number  int64  
 2   Invoice Date    object 
 3   SKU             int64  
 4   Category        object 
 5   Product         object 
 6   QTY             int64  
 7   Size            object 
 8   Gross Margin    float64
 9   Sales           float64
dtypes: float64(2), int64(4), object(4)
memory usage: 330.5+ MB


In [6]:
sum(transaction_data.duplicated()) # The data do not contain duplicated records.

0

Brief Descriptions of Input variables:
1. **StoreID**  (numeric) ID number for each store of Round Roasters
2. **Invoice Number** (numeric) 
3. **Invoice Date** (numeric) 
4. **SKU** (numeric): SKU is abbreviation of Stock Keeping Unit. It is a scannable bar code, most often seen printed on product labels in a retail store. The label allows vendors to automatically track the movement of inventory.
5. **Category** (Categorical: 'Espresso', 'Pastry', 'Milk', 'Coffee', 'Sandwich', 'Wine' ): Product category
6. **Product** (Categorical: 'Mocha', 'Croissant', 'Espresso', 'Latte', 'Cappuccino', 'Hot Chocolate', 'Danish', 'Drip', 'Caprese', 'Pinot Grigio', 'Cabernet Sauvignon', 'Croque Madame' ): Name of product purchased
7. **QTY** (numeric) Quantity of products purchased
8. **Size** (Categorical:  'S', 'M', 'L'): Size of each product purchased 
9. **Gross Margin** (numeric): Gross Margin of each product
10. **Sales** (numeric): Sales price of each product

In [7]:
# Check the transaction records with missing values
null_data = transaction_data[transaction_data.isnull().any(axis=1)]
null_data 

,StoreID,Invoice Number,Invoice Date,SKU,Category,Product,QTY,Size,Gross Margin,Sales
1,10018,16296643,2015-01-21,2001,Pastry,Croissant,1,NaN,1.10,2.75
6,10018,16297717,2015-01-21,2001,Pastry,Croissant,4,NaN,4.40,11.00
9,10018,16298672,2015-01-21,2001,Pastry,Croissant,1,NaN,1.10,2.75
24,10018,16300868,2015-01-21,2001,Pastry,Croissant,2,NaN,2.20,5.50
25,10018,16300868,2015-01-21,2002,Pastry,Danish,1,NaN,1.05,3.50
...,...,...,...,...,...,...,...,...,...,...
4332273,9968,45357910,2016-07-20,2001,Pastry,Croissant,4,NaN,4.40,11.00
4332278,9968,45358218,2016-07-20,2001,Pastry,Croissant,3,NaN,3.30,8.25
4332283,9968,45358287,2016-07-20,2001,Pastry,Croissant,3,NaN,3.30,8.25
4332287,9968,45358830,2016-07-21,2001,Pastry,Croissant,4,NaN,4.40,11.00


In [8]:
# It seems that the column "Size" includes the most missing values. Let us check how many.
print(sum(transaction_data.Size.isnull())) 

942725


In [9]:
transaction_data.Category.value_counts() # Original Data 

Espresso    3198571
Pastry       921588
Milk         180229
Sandwich      19621
Coffee        10808
Wine           1516
Name: Category, dtype: int64

In [10]:
null_data.Category.value_counts() # null_data

Pastry      921588
Sandwich     19621
Wine          1516
Name: Category, dtype: int64

<a id='assess_transaction'></a>
#### Data Assessment (Transaction Data)
First, I loaded the transaction dataset. It has 4,332,333 datapoints and 10 features. The column: "Size" in the large dataset contains
942,725 missing values. Product categories: "Pastry", "Sandwich", and "Wine" has the missing value of Size. Products in those categories probably have one-size only. In addition, I have checked the datatypes of each field. I made the following notes to clean the data. 
- (1) Some column names contain 'space( )' between the words. It should be replaced with 'underscore' for convenient usage of pandas
- (2) The columns **"StoreID", "Invoice Number", "SKU"**,  should have an "object" type, rather than "int64". However, I kept the initial datatype of StoreID for the sake of convenience. 
- (3) **'Invoice Date'** should have datetime datatype.  
- (4) The missing values in the column **'Size'** could be imputed "One-Size Only".<br>
- (5) Since the dataset includs daily transaction histories. it is very large. The Gross Margin and Sales can be aggregated per week. 
Based on the data assessments, the initial transaction dataset is modified as below. 

<a id='cleaning_transaction'></a>
#### Data Cleaning (Transaction Data)

In [11]:
# Make a copy of the original dataset. It works as a checkpoint.
transaction_cleaned = transaction_data.copy() 
# (1) Change the column names with underscore
transaction_cleaned.columns = transaction_cleaned.columns.str.replace(" ", "_")

In [12]:
# (2)-(3) Change the datatype of the columns 
transaction_cleaned.Invoice_Number=transaction_cleaned.Invoice_Number.apply(str)
transaction_cleaned.SKU=transaction_cleaned.SKU.apply(str)
transaction_cleaned.Invoice_Date=pd.to_datetime(transaction_cleaned.Invoice_Date)

In [13]:
# (4) Fill up the missing datapoints with "One-Size"
values = {'Size': 'One-Size'} 
transaction_cleaned.fillna(value=values, inplace = True); 

In [14]:
transaction_cleaned.info() # All datatypes are properly updated.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4332333 entries, 0 to 4332332
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   StoreID         int64         
 1   Invoice_Number  object        
 2   Invoice_Date    datetime64[ns]
 3   SKU             object        
 4   Category        object        
 5   Product         object        
 6   QTY             int64         
 7   Size            object        
 8   Gross_Margin    float64       
 9   Sales           float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 330.5+ MB


In [15]:
print(sum(transaction_cleaned.Size.isnull())) # No Missing values 

0


### 2. All Stores Data

In [16]:
all_stores.head() # Glance at the dataset(1) 

,StoreID,Sq_Ft,AvgMonthSales,Right_Name,Phone Number,Street Combined,Street 1,Street 2,Street 3,City,State,Postal Code,Region,Country,Coordinates,Latitude,Longitude,Timezone,Current Timezone Offset,Olson Timezone
0,10018,1183,18000,Bellflower & Spring,562-420-1317,"2890 N Bellflower Blvd, #A-1, The Los Altos Ma...",2890 N Bellflower Blvd,#A-1,The Los Altos Marketplace,Long Beach,CA,908151125,West,US,"(33.8085823059082, -118.124931335449)",33.808582,-118.124931,Pacific Standard Time,-480,GMT-08:00 America/Los_Angeles
1,10068,1198,16000,"Foothill & Boston, La Crescenta",818-541-7693,"3747 Foothill Boulevard, A",3747 Foothill Boulevard,A,NaN,La Cresenta,CA,912141700,West,US,"(34.2375450134277, -118.26114654541)",34.237545,-118.261147,Pacific Standard Time,-480,GMT-08:00 America/Los_Angeles
2,10118,1204,13000,Magic Mountain & Tourney,661-260-0844,"25349 Wayne Mills Place, Tourney Retail Plaza",25349 Wayne Mills Place,NaN,Tourney Retail Plaza,Valencia,CA,913551827,West,US,"(34.4237022399902, -118.579261779785)",34.423702,-118.579262,Pacific Standard Time,-480,GMT-08:00 America/Los_Angeles
3,10168,1195,19000,Alameda & Shelton,(818) 557-6604,"1190 Alameda Avenue, Suite G-2",1190 Alameda Avenue,Suite G-2,NaN,Burbank,CA,915062806,West,US,"(34.1625328063965, -118.314529418945)",34.162533,-118.314529,Pacific Standard Time,-480,GMT-08:00 America/Los_Angeles
4,10218,1193,15000,Victoria Gardens,909-646-8562,"12466 North Main Street, Space 3340, Victoria ...",12466 North Main Street,Space 3340,Victoria Gardens,Rancho Cucamonga,CA,917398886,West,US,"(34.1120910644531, -117.533462524414)",34.112091,-117.533462,Pacific Standard Time,-480,GMT-08:00 America/Los_Angeles


In [17]:
all_stores.info() # Glance at the dataset(2) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   StoreID                  133 non-null    int64  
 1   Sq_Ft                    133 non-null    int64  
 2   AvgMonthSales            133 non-null    int64  
 3   Right_Name               133 non-null    object 
 4   Phone Number             130 non-null    object 
 5   Street Combined          133 non-null    object 
 6   Street 1                 133 non-null    object 
 7   Street 2                 37 non-null     object 
 8   Street 3                 25 non-null     object 
 9   City                     133 non-null    object 
 10  State                    133 non-null    object 
 11  Postal Code              133 non-null    int64  
 12  Region                   133 non-null    object 
 13  Country                  133 non-null    object 
 14  Coordinates              1

> **Note:** The last datasets are the list of all Round Roaster stores. The columns 'Phone Number' and 'Street 2, 3' include missing values, but they are not relevant to the analysis. 

### 3. Treatment Stores Data

In [18]:
treatment_stores.head() # Glance at the dataset(1) 

,StoreID,Sq_Ft,AvgMonthSales,Phone Number,Street Combined,Street 1,Street 2,Street 3,City,Postal Code,Region,Country,Coordinates,Latitude,Longitude,Timezone,Current Timezone Offset,Olson Timezone,Name,Right_State
0,1664,1475,11000,8478428048,"101 W. Main St., Barrington Village Center",101 W. Main St.,NaN,Barrington Village Center,Barrington,60010,Central,US,"(42.1540565490723, -88.1362915039063)",42.154057,-88.136291,Central Standard Time,-360,GMT-06:00 America/Chicago,Barrington,IL
1,1675,1472,15000,8472531188,90 East Northwest Highway,90 East Northwest Highway,NaN,NaN,Mount Prospect,60056,Central,US,"(42.0633544921875, -87.9355773925781)",42.063354,-87.935577,Central Standard Time,-360,GMT-06:00 America/Chicago,Northwest Hwy & Elmhurst Rd,IL
2,1696,1471,10000,2242232528,1261 East Higgins Road,1261 East Higgins Road,NaN,NaN,Schaumburg,60173,Central,US,"(42.039363861084, -88.048828125)",42.039364,-88.048828,Central Standard Time,-360,GMT-06:00 America/Chicago,Higgins & Meacham,IL
3,1700,1465,15000,(224) 500-9575,17 W 633 Roosevelt Road,17 W 633 Roosevelt Road,NaN,NaN,Oakbrook Terrace,60181,Central,US,"(41.8600807189941, -87.9739685058594)",41.860081,-87.973969,Central Standard Time,-360,GMT-06:00 America/Chicago,Roosevelt & Summit,IL
4,1712,1456,19000,708-403-1461,15858 South LaGrange Road,15858 South LaGrange Road,NaN,NaN,Orland Park,60462,Central,US,"(41.603816986084, -87.8534317016602)",41.603817,-87.853432,Central Standard Time,-360,GMT-06:00 America/Chicago,159th & LaGrange,IL


In [19]:
treatment_stores.info() # Glance at the dataset(2) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   StoreID                  10 non-null     int64  
 1   Sq_Ft                    10 non-null     int64  
 2   AvgMonthSales            10 non-null     int64  
 3   Phone Number             9 non-null      object 
 4   Street Combined          10 non-null     object 
 5   Street 1                 10 non-null     object 
 6   Street 2                 1 non-null      float64
 7   Street 3                 1 non-null      object 
 8   City                     10 non-null     object 
 9   Postal Code              10 non-null     int64  
 10  Region                   10 non-null     object 
 11  Country                  10 non-null     object 
 12  Coordinates              10 non-null     object 
 13  Latitude                 10 non-null     float64
 14  Longitude                10 n

In [20]:
treatment_stores.StoreID.unique() # Store ID of treatments 

array([1664, 1675, 1696, 1700, 1712, 2288, 2293, 2301, 2322, 2341],
      dtype=int64)

> **Note:** 10 Stores with Store ID (1664, 1675, 1696, 1700, 1712, 2288, 2293, 2301, 2322, 2341) are chosen as treatment group by the company. The columns 'Phone Number' and 'Street 2, 3' include missing values, but they are not relevant to the analysis. 

### 4. Tidiness

To get a handle on the problem, I modified the given datasets. To make data set tidy provides a way to organise data values within a dataset.<br> [Reference: Tidy Data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html) 

**Assessment of Tidiness**
1. Tranaction dataset contains too much information. Only the relevant period of records should be filtered. 
2. Since the test ran for the week-based periods, the data should be aggregated at the weekly level accordingly. 
3. "Traffic records" and "Gross Margin and Sales" information should be separated from the Tranaction dataset.
4. Both "treatment_stores" and "control_stores" contain stores information. Thus it is better to combine them after labeling. 

#### Data Cleaning (Tidiness) 

1. Filter datapoints  

> To use as control variables, we need to create trend and seasonality variables.  To do this, I need at least 52 weeks (ca. 1 year) of data, plus the number of weeks I select to calculate trend, before the beginning of the test start date. For example, when we use 6 weeks to calculate the trend, then we need 58 (= 52 + 6) weeks prior to the test start date. In this project, the company asked me to analyse 12 weeks to calculate trend, so I need 64 weeks of data prior to the test start date. Additionally, since the test lasts for 12 weeks, this indicates I need a total 76 weeks (12 + 12 + 52 weeks) of data. Therefore, I filter 76 weeks from the end of the test period 2016-July-21. 

In [21]:
# Import the relevant libraries 
import datetime 
from datetime import timedelta 

In [22]:
# Calculate the start date of test
test_end_date = '2016-07-22'
end_date = datetime.datetime.strptime(test_end_date, '%Y-%m-%d') 
start_date = end_date - datetime.timedelta(weeks=76)
print('Test start date is', start_date)

Test start date is 2015-02-06 00:00:00


In [23]:
# Using the start and end date, the previous dateset is filtered. 
filtered_df = transaction_cleaned.query('Invoice_Date >="2015-02-06" and Invoice_Date <"2016-07-22"')

In [24]:
filtered_df.head()

,StoreID,Invoice_Number,Invoice_Date,SKU,Category,Product,QTY,Size,Gross_Margin,Sales
628,10018,16551067,2015-02-06,1013,Espresso,Cappuccino,2,L,4.7880,7.98
629,10018,16551067,2015-02-06,1022,Espresso,Espresso,1,M,1.4950,2.99
630,10018,16551067,2015-02-06,1023,Espresso,Espresso,3,L,4.7850,9.57
631,10018,16551067,2015-02-06,1041,Espresso,Mocha,1,S,1.7955,3.99
632,10018,16551067,2015-02-06,1042,Espresso,Mocha,3,M,6.0615,13.47


2. Aggregate the transaction data weekly-based  

In [25]:
start_date = '2015-02-06'
filtered_df.loc[:, 'DateTimeDiff'] = filtered_df['Invoice_Date'] - datetime.datetime.strptime(start_date, '%Y-%m-%d') 

C:\Users\wonso\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\wonso\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [26]:
# Create the Week number 
import datetime as dt 
filtered_df['DateTimeDiff']=filtered_df['DateTimeDiff'].dt.days 
filtered_df['Week']=filtered_df['DateTimeDiff']//7 +1 
days_to_shift = pd.TimedeltaIndex(7*(filtered_df['Week']-1), unit="D")
filtered_df['Week_Start'] = datetime.datetime.strptime(start_date, '%Y-%m-%d') + days_to_shift 
filtered_df['Week_End']  = filtered_df['Week_Start'] + timedelta(days=6) 

C:\Users\wonso\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\wonso\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\wonso\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [27]:
filtered_df.head()

,StoreID,Invoice_Number,Invoice_Date,SKU,Category,Product,QTY,Size,Gross_Margin,Sales,DateTimeDiff,Week,Week_Start,Week_End
628,10018,16551067,2015-02-06,1013,Espresso,Cappuccino,2,L,4.7880,7.98,0,1,2015-02-06,2015-02-12
629,10018,16551067,2015-02-06,1022,Espresso,Espresso,1,M,1.4950,2.99,0,1,2015-02-06,2015-02-12
630,10018,16551067,2015-02-06,1023,Espresso,Espresso,3,L,4.7850,9.57,0,1,2015-02-06,2015-02-12
631,10018,16551067,2015-02-06,1041,Espresso,Mocha,1,S,1.7955,3.99,0,1,2015-02-06,2015-02-12
632,10018,16551067,2015-02-06,1042,Espresso,Mocha,3,M,6.0615,13.47,0,1,2015-02-06,2015-02-12


3. Separate the information: "Traffic records" and "Gross Margin and Sales" 

- **Weekly Traffic Records**

In [28]:
# The Number of Unique Invoices per Store and per Week
filtered_df.groupby(['StoreID', 'Week',"Week_Start", "Week_End"])['Invoice_Number'].nunique() 

StoreID  Week  Week_Start  Week_End  
1508     1     2015-02-06  2015-02-12    183
         2     2015-02-13  2015-02-19    273
         3     2015-02-20  2015-02-26    153
         4     2015-02-27  2015-03-05    210
         5     2015-03-06  2015-03-12    239
                                        ... 
12836    72    2016-06-17  2016-06-23    212
         73    2016-06-24  2016-06-30    248
         74    2016-07-01  2016-07-07    188
         75    2016-07-08  2016-07-14    311
         76    2016-07-15  2016-07-21    312
Name: Invoice_Number, Length: 10108, dtype: int64

In [29]:
# Change the above 'groupby' output into DataFrame
traffic_df=filtered_df.reset_index().groupby(['StoreID', 'Week',"Week_Start", "Week_End"],as_index=False)['Invoice_Number'].nunique()
traffic_df.head()

,StoreID,Week,Week_Start,Week_End,Invoice_Number
0,1508,1,2015-02-06,2015-02-12,183
1,1508,2,2015-02-13,2015-02-19,273
2,1508,3,2015-02-20,2015-02-26,153
3,1508,4,2015-02-27,2015-03-05,210
4,1508,5,2015-03-06,2015-03-12,239


- **Gross Margin and Sales**

In [30]:
# Gross Margin and Sales per Store and per Week
filtered_df.groupby(['StoreID', 'Week',"Week_Start", "Week_End"])['Gross_Margin', 'Sales'].sum()

C:\Users\wonso\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


Gross_Margin    Sales
StoreID Week Week_Start Week_End                         
1508    1    2015-02-06 2015-02-12     1429.4390  3124.61
        2    2015-02-13 2015-02-19     2031.2580  4360.54
        3    2015-02-20 2015-02-26     1022.0430  2217.27
        4    2015-02-27 2015-03-05     1372.2495  2996.95
        5    2015-03-06 2015-03-12     1698.8590  3704.73
...                                          ...      ...
12836   72   2016-06-17 2016-06-23     1326.5520  2781.26
        73   2016-06-24 2016-06-30     1728.0585  3654.48
        74   2016-07-01 2016-07-07     1339.2400  2861.75
        75   2016-07-08 2016-07-14     2178.2930  4601.37
        76   2016-07-15 2016-07-21     1905.1625  4053.40

[10108 rows x 2 columns]

In [31]:
# Change the above 'groupby' output into DataFrame
weekly_sales_df=filtered_df.reset_index().groupby(['StoreID', 'Week',"Week_Start", "Week_End"],as_index=False)['Gross_Margin', 'Sales'].sum()
weekly_sales_df.head()

C:\Users\wonso\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,StoreID,Week,Week_Start,Week_End,Gross_Margin,Sales
0,1508,1,2015-02-06,2015-02-12,1429.4390,3124.61
1,1508,2,2015-02-13,2015-02-19,2031.2580,4360.54
2,1508,3,2015-02-20,2015-02-26,1022.0430,2217.27
3,1508,4,2015-02-27,2015-03-05,1372.2495,2996.95
4,1508,5,2015-03-06,2015-03-12,1698.8590,3704.73


4. Combine "treatment_stores" and "control_stores" after labeling

In [32]:
treatment_stores['Group'] = 'treatment'
all_stores['Group'] = 'control'

In [33]:
treatment_stores.columns

Index(['StoreID', 'Sq_Ft', 'AvgMonthSales', 'Phone Number', 'Street Combined',
       'Street 1', 'Street 2', 'Street 3', 'City', 'Postal Code', 'Region',
       'Country', 'Coordinates', 'Latitude', 'Longitude', 'Timezone',
       'Current Timezone Offset', 'Olson Timezone', 'Name', 'Right_State',
       'Group'],
      dtype='object')

In [34]:
all_stores.columns 

Index(['StoreID', 'Sq_Ft', 'AvgMonthSales', 'Right_Name', 'Phone Number',
       'Street Combined', 'Street 1', 'Street 2', 'Street 3', 'City', 'State',
       'Postal Code', 'Region', 'Country', 'Coordinates', 'Latitude',
       'Longitude', 'Timezone', 'Current Timezone Offset', 'Olson Timezone',
       'Group'],
      dtype='object')

In [35]:
# Rename the columns with different names 
treatment_stores.rename(columns={"Right_State": "State"}, inplace = True)
all_stores.rename(columns={"Right_Name": "Name"}, inplace = True)

In [36]:
# Drop the unnecessary columns from treatment_stores and control_stores dataframes 
treatment_stores.drop(['Phone Number', 'Street Combined', 'Street 1', 'Street 2', 'Street 3', 'Postal Code', 'Country', 'Coordinates', 'Latitude',
       'Longitude', 'Timezone', 'Current Timezone Offset', 'Olson Timezone' ],axis =1, inplace = True)
all_stores.drop(['Phone Number', 'Street Combined', 'Street 1', 'Street 2', 'Street 3',  'Postal Code', 'Country', 'Coordinates', 'Latitude',
       'Longitude', 'Timezone', 'Current Timezone Offset', 'Olson Timezone' ],axis =1, inplace =True)


In [37]:
stores_df =  all_stores.append(treatment_stores, ignore_index=True)

In [38]:
stores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   StoreID        143 non-null    int64 
 1   Sq_Ft          143 non-null    int64 
 2   AvgMonthSales  143 non-null    int64 
 3   Name           143 non-null    object
 4   City           143 non-null    object
 5   State          143 non-null    object
 6   Region         143 non-null    object
 7   Group          143 non-null    object
dtypes: int64(3), object(5)
memory usage: 9.1+ KB


> After cleaning up the data, we have three datasets. 
> - **traffic_df:** It contains the number of traffic information. The number of traffic is aggregated per week per store. 
> - **weekly_sales_df:** It contains weekly gross margin and sales information per store. 
> - **stores_df:** It includes all stores of the company. Each store is labelled  as treatment or control. 
> Each store represents each unit. Since we have relatively small size of sample units (133 Stores in total), we need to select matched-pair design for this analysis, rather than randomized design. 


In [39]:
traffic_df.to_csv('traffic_df.csv', index = False)
weekly_sales_df.to_csv('weekly_sales_df.csv', index = False)
stores_df.to_csv('stores_df.csv', index = False)